In [ ]:
import ROOT
from ROOT import gROOT, TString
from ROOT import TCanvas, TLine, TGaxis, TGraph, gPad, TF1, TF2, kRed, TMultiGraph, TLegend, gStyle, TPaveStats, TStyle, TText, TList, TLatex, TGraphErrors, TFile, TTree
import json
import math
from array import array
import numpy as np
import pandas as pd

In [ ]:
sigma_factor = (6.2*10e-3)/1.5

def HVCurrentDataFrame(scanIds):
    current_top = []    
    current_bot = []    
    HV_top = []    
    HV_bot = []
    eff = []
    eff_error = []
    muon_stream = []
    gamma_stream = []
    muon_CS = []
    gamma_CS = []
    muon_CM = []
    gamma_CM = []
    muon_CS_err = []
    gamma_CS_err = []
    muon_CM_err = []
    gamma_CM_err = []
    noiseGammaRate = []
    deltaV_top = []
    deltaV_bot = []
    deltaV_top_err = []
    deltaV_bot_err = []
    for scanId in scanIds:

        
        !echo "None" >> counter.txt
        !rm counter.txt
        path = "Scans_2024/Scan_00"+str(scanId)+"/*_HV*_CAEN.root"
        !ls {path} >> counter.txt
        counter = np.loadtxt("counter.txt", dtype=str)
        N = len(counter)+1

        
        for i in range(1,N):
            jsonFile = open("Scans_2024/Scan_00"+str(scanId)+"/ANALYSIS/KODELE/HV"+str(i)+"/output.json")
            jsonFile = json.load(jsonFile)
            histFile  = ROOT.TFile.Open("Scans_2024/Scan_00"+str(scanId)+"/Scan00"+str(scanId)+"_HV"+str(i)+"_CAEN.root","READ")
            hist_current_bot = histFile.Get("Imon_KODELE-BOT")
            hist_current_top = histFile.Get("Imon_KODELE-TOP")
            hist_HV_bot = histFile.Get("HVeff_KODELE-BOT")
            hist_HV_top = histFile.Get("HVeff_KODELE-TOP")
            eff.append(jsonFile['output_parameters']['efficiencyMuon_corrected'])
            eff_error.append(jsonFile['output_parameters']['efficiencyMuon_corrected_err'])
            muon_stream.append(jsonFile['output_parameters']['muonStreamerProbability'])
            gamma_stream.append(jsonFile['output_parameters']['gammaStreamerProbability'])
            muon_CS.append(jsonFile['output_parameters']['muonCLS'])
            gamma_CS.append(jsonFile['output_parameters']['gammaCLS'])
            muon_CM.append(jsonFile['output_parameters']['muonCMP'])
            gamma_CM.append(jsonFile['output_parameters']['gammaCMP'])
            muon_CS_err.append(jsonFile['output_parameters']['muonCLS_err'])
            gamma_CS_err.append(jsonFile['output_parameters']['gammaCLS_err'])
            muon_CM_err.append(jsonFile['output_parameters']['muonCMP_err'])
            gamma_CM_err.append(jsonFile['output_parameters']['gammaCMP_err'])
            noiseGammaRate.append(jsonFile['output_parameters']['noiseGammaRate'])
            current_bot.append(hist_current_bot.GetMean())
            current_top.append(hist_current_top.GetMean())
            HV_top.append(hist_HV_top.GetMean())
            HV_bot.append(hist_HV_bot.GetMean())
            deltaV_top.append(hist_current_top.GetMean()*1.5)
            deltaV_bot.append(hist_current_bot.GetMean()*1.5)
            deltaV_top_err.append(hist_current_top.GetMean()*6.2*10e-3)
            deltaV_bot_err.append(hist_current_bot.GetMean()*6.2*10e-3)
            histFile.Close()

    
    DataSet = {'HV_top': HV_top, 
               'HV_bot': HV_bot, 
               'current_top': current_top, 
               'current_bot': current_bot,
               'muon_stream': muon_stream,
               'gamma_stream': gamma_stream,
               'muon_CM': muon_CM,
               'gamma_CM': gamma_CM,
               'muon_CS': muon_CS,
               'gamma_CS': gamma_CS,
               'muon_CM_err': muon_CM_err,
               'gamma_CM_err': gamma_CM_err,
               'muon_CS_err': muon_CS_err,
               'gamma_CS_err': gamma_CS_err,
               'efficiency': eff,
               'eff_error': eff_error,
               'noiseGammaRate': noiseGammaRate,
               'deltaV': (np.array(deltaV_top)*np.array(deltaV_bot)+np.full_like(np.array(deltaV_bot), 0.00000001))/(np.array(deltaV_top)+np.array(deltaV_bot)+np.full_like(np.array(deltaV_bot), 0.00000001)),
               }
    
    DataSet = pd.DataFrame(data=DataSet)
    DataSet['current'] = DataSet['current_top'] + DataSet['current_bot']
    DataSet['deltaV_err'] = DataSet['deltaV']* sigma_factor *np.sqrt( (1 + (np.sqrt(DataSet['current_top']**2 + DataSet['current_bot']**2 ) / ( DataSet['current_top']+DataSet['current_bot']  ) )**2 ) ) 
    return DataSet

In [ ]:
def FeaturesDataFrame(scanIds):
    
    current_top = []    
    current_bot = []    
    eff = []
    eff_error = []
    muon_stream = []
    gamma_stream = []
    muon_CS = []
    gamma_CS = []
    muon_CM = []
    gamma_CM = []
    muon_CS_err = []
    gamma_CS_err = []
    muon_CM_err = []
    gamma_CM_err = []
    current_bot = []
    current_top = []
    HV_top = []
    HV_bot = []
    noiseGammaRate = []
    noiseGammaRate_err = []

    
    for scanId in scanIds:
        print("scanID: ", scanIds[scanId][0])
        N = "1"
        
        jsonFile = open("Scans_2024/Scan_00"+str(scanIds[scanId][0])+"/ANALYSIS/KODELE/HV"+str(N)+"/output.json")
        histFile  = ROOT.TFile.Open("Scans_2024/Scan_00"+str(scanIds[scanId][0])+"/Scan00"+str(scanIds[scanId][0])+"_HV"+str(N)+"_CAEN.root","READ")
        hist_current_bot = histFile.Get("Imon_KODELE-BOT")
        hist_current_top = histFile.Get("Imon_KODELE-TOP")
        hist_HV_bot = histFile.Get("HVeff_KODELE-BOT")
        hist_HV_top = histFile.Get("HVeff_KODELE-TOP")
        jsonFile = json.load(jsonFile)
        nTrig = jsonFile['output_parameters']['nTrig']
        eff.append(100*jsonFile['output_parameters']['efficiencyMuon_corrected'])
        eff_error.append(jsonFile['output_parameters']['efficiencyMuon_corrected_err'])
        muon_stream.append(jsonFile['output_parameters']['muonStreamerProbability'])
        gamma_stream.append(jsonFile['output_parameters']['gammaStreamerProbability'])
        muon_CS.append(jsonFile['output_parameters']['muonCLS'])
        gamma_CS.append(jsonFile['output_parameters']['gammaCLS'])
        muon_CM.append(jsonFile['output_parameters']['muonCMP'])
        gamma_CM.append(jsonFile['output_parameters']['gammaCMP'])
        muon_CS_err.append(jsonFile['output_parameters']['muonCLS_err'])
        gamma_CS_err.append(jsonFile['output_parameters']['gammaCLS_err'])
        muon_CM_err.append(jsonFile['output_parameters']['muonCMP_err'])
        gamma_CM_err.append(jsonFile['output_parameters']['gammaCMP_err'])
        noiseGammaRate.append(jsonFile['output_parameters']['noiseGammaRate'])
        noiseGammaRate_err.append(jsonFile['output_parameters']['noiseGammaRate_err'])
        current_bot.append(hist_current_bot.GetMean())
        current_top.append(hist_current_top.GetMean())
        HV_top.append(hist_HV_top.GetMean())
        HV_bot.append(hist_HV_bot.GetMean())
        
    DataSet = {'current_top': current_top, 
               'current_bot': current_bot,
               'HV_top': HV_top,
               'HV_bot': HV_bot,
               'muon_stream': muon_stream,
               'gamma_stream': gamma_stream,
               'muon_CM': muon_CM,
               'gamma_CM': gamma_CM,
               'muon_CS': muon_CS,
               'gamma_CS': gamma_CS,
               'muon_CM_err': muon_CM_err,
               'gamma_CM_err': gamma_CM_err,
               'muon_CS_err': muon_CS_err,
               'gamma_CS_err': gamma_CS_err,
               'efficiency': eff,
               'eff_error': eff_error,
               'noiseGammaRate': noiseGammaRate,
               'noiseGammaRate_err': noiseGammaRate_err
              }
        
    DataSet = pd.DataFrame(data=DataSet)
    DataSet['current'] = DataSet['current_top'] + DataSet['current_bot']
    
    return DataSet

In [ ]:
# Dicionários de Scans de 2024. Mude os scans conforme necessário.
scans = {
    'STDMX_OFF': ['6001'],
    'STDMX_22':  ['6009'],
    'STDMX_10':  ['6011'],
    'STDMX_6.9': ['6007'],
    'STDMX_3.3': ['6014'],
    'STDMX_2.2': ['5999'],
    'STDMX_1':   ['6005'],
    
    '30CO205SF6_OFF': ['5981'],
    '30CO205SF6_22':  ['5985'],
    '30CO205SF6_10':  ['5979'],
    '30CO205SF6_6.9': ['5975'],
    '30CO205SF6_3.3': ['5977'],
    '30CO205SF6_2.2': ['5987'],
    '30CO205SF6_1':   ['5983'],
    
    '30CO2_OFF':      ['6019'],
    '30CO2_22':       ['6021'],
    '30CO2_10':       ['6015'],
    '30CO2_6.9':      ['6016'],
    '30CO2_3.3':      ['6023'],
    '30CO2_2.2':      ['6025'],
    '30CO2_1':        ['6027'],
    
    '40CO2_OFF':      ['5959'],
    '40CO2_22':       ['5953'],
    '40CO2_10':       ['5961'],
    '40CO2_6.9':      ['5951'],
    '40CO2_4.6':      ['5947'],
    '40CO2_3.3':      ['5957'],
    '40CO2_2.2':      ['5963'],
    '40CO2_1':        ['5955']
}

#Scans no WP
Scans_STDMX = {
    
    'STDMX_OFF':  ['6004'],
    'STDMX_22':   ['6010'],
    'STDMX_10':   ['6012'],
    'STDMX_6.9':  ['6008'],
    'STDMX_3.3':  ['6003'],
    'STDMX_2.2':  ['6000'],
    'STDMX_1':    ['6006'],
    
    '30CO2_OFF':  ['6029'],
    '30CO2_22':   ['6022'],
    '30CO2_10':   ['6020'],
    '30CO2_3.3':  ['6024'],
    '30CO2_2.2':  ['6026'],
    '30CO2_1':    ['6028'],
    
    '30CO205SF6_OFF':  ['5982'],
    '30CO205SF6_22':   ['5986'],
    '30CO205SF6_10':   ['5980'],
    '30CO205SF6_6.9':  ['5973'],
    '30CO205SF6_3.3':  ['5978'],
    '30CO205SF6_2.2':  ['5988'],
    '30CO205SF6_1':    ['5984'],

    '40CO2_OFF':  ['5960'],
    '40CO2_22':   ['5954'],
    '40CO2_10':   ['5962'],
    '40CO2_6.9':  ['5952'],
    '40CO2_4.6':  ['5948'],
    '40CO2_3.3':  ['5958'],
    '40CO2_2.2':  ['5964'],
    '40CO2_1':    ['5956']
}

In [ ]:
for scan in scans:
    print(scan, ': ')
    print(HVCurrentDataFrame(scans[scan]))

In [ ]:
for scan in scans:
    df = HVCurrentDataFrame(scans[scan])  
    filename = f"{scan}.csv"  
    df.to_csv(filename, index=False)  
    print(f"Arquivo salvo: {filename}")

In [ ]:
import pandas as pd

for scan in Scans_STDMX:
    df = FeaturesDataFrame({scan: Scans_STDMX[scan]})  
    filename = f"{scan}_WP.csv"  
    df.to_csv(filename, index=False)  
    print(f"Arquivo salvo: {filename}")